In [19]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random
import torch.utils.data as Data
from torch import nn
import torch.optim as optim

## 整体实现

In [38]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1, (num_examples, num_inputs)), dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)

batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))

nn.init.normal_(net[0].weight, mean=0, std=0.01)
nn.init.constant_(net[0].bias, val=0)

optimizer = optim.SGD(net.parameters(), lr=0.03)

num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零 等同于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))
    
#比较学习到的模型参数和真实的模型参数
#从net中获得需要的层，并访问其权重与偏差
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

epoch 1, loss: 0.000407
epoch 2, loss: 0.000057
epoch 3, loss: 0.000083
[2, -3.4] Parameter containing:
tensor([[ 1.9992, -3.4003]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1993], requires_grad=True)


## 分步解释

#### 生成数据集

In [ ]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.from_numpy(np.random.normal(0,1, (num_examples, num_inputs)))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.from_numpy(np.random.normal(0, 0.01, size=labels.size()))

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

loss = nn.MSELoss()

#### 读取数据集

In [3]:
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

#### 定义模型

In [11]:
# class LinearNet(nn.Module):
#     def __init__(self, n_feature):
#         super(LinearNet, self).__init__()
#         self.linear = nn.Linear(n_feature, 1)
#     def forword(self, x):
#         y = self.linear(x)
#         return y
# net = LinearNet(num_inputs)
# print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [15]:
# 还可以用nn.Sequential搭建网络
#Sequential是一个有序的容器，
#网络层将按照在传入Sequential的顺序依次被添加到计算图中

#写法一：
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    #后面可以继续添加其他层
    )

#写法二：
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
#net.add_module......

#写法三：
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
    ('linear', nn.Linear(num_inputs, 1))
    #......
    ]))
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [13]:
# 可以通过net.paramters()来查看模型所有的可学习参数
# 此函数将返回一个生成器
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.5410,  0.5893]], requires_grad=True)
Parameter containing:
tensor([-0.6557], requires_grad=True)


#### 初始化模型参数

In [28]:
nn.init.normal_(net[0].weight, mean=0, std=0.01)
nn.init.constant_(net[0].bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

#### 定义损失函数

In [29]:
loss = nn.MSELoss()

#### 定义优化算法

In [30]:
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [23]:
# 我们还可以为不同网络设置不同的学习率
# optimizer = optim.SGD([
#     # 如果对某个参数不指定学习率， 就使用最外层的默认学习率
#     {'params': net.subnet1.parameters()}, # lr=0.03
#     {'params': net.subnet2.parameters(), 'lr': 0.01}
#     ], lr=0.03)

In [25]:
# 调整学习率
# for param_group in optimizer.param_groups:
#     print(param_group)
#     param_group['lr'] *= 0.1

#### 训练模型

In [31]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零 等同于net.zero_grad()
        l.backword()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'